<a href="https://colab.research.google.com/github/cursosLabra/pruebawebsem/blob/main/Introducci%C3%B3n_a_Web_Sem%C3%A1ntica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a RDF

In [ ]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


En esta sección, se va a introducir RDF. Para ello, se requiere instalar la  librería [rdflib](https://rdflib.readthedocs.io/).

A partir de ahí, se pueden crear grafos importando las clases correspondientes de rdflib.

In [ ]:
from rdflib import Graph

In [ ]:
g = Graph()

In [ ]:
g.parse("https://raw.githubusercontent.com/cursosLabra/pruebawebsem/main/example1.ttl", format="turtle")

<Graph identifier=N4386f932b84e49308fbc9107e9db2b73 (<class 'rdflib.graph.Graph'>)>

In [ ]:
for contador, t in enumerate(g):
  print(f"Tripleta {contador}: sujeto = {t[0]}, predicado = {t[1]}, objeto = {t[2]}") 

Tripleta 0: sujeto = http://example.org/carol, predicado = https://schema.org/knows, objeto = http://example.org/alice
Tripleta 1: sujeto = http://example.org/carol, predicado = https://schema.org/knows, objeto = http://example.org/bob
Tripleta 2: sujeto = http://example.org/bob, predicado = https://schema.org/knows, objeto = http://example.org/carol
Tripleta 3: sujeto = http://example.org/alice, predicado = https://schema.org/knows, objeto = http://example.org/bob


In [ ]:
print(g.serialize(format="nt"))

<http://example.org/carol> <https://schema.org/knows> <http://example.org/alice> .
<http://example.org/carol> <https://schema.org/knows> <http://example.org/bob> .
<http://example.org/bob> <https://schema.org/knows> <http://example.org/carol> .
<http://example.org/alice> <https://schema.org/knows> <http://example.org/bob> .



In [ ]:
print(g.serialize(format="xml"))

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
   xmlns:schema="https://schema.org/"
>
  <rdf:Description rdf:about="http://example.org/carol">
    <schema:knows rdf:resource="http://example.org/bob"/>
    <schema:knows rdf:resource="http://example.org/alice"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://example.org/bob">
    <schema:knows rdf:resource="http://example.org/carol"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://example.org/alice">
    <schema:knows rdf:resource="http://example.org/bob"/>
  </rdf:Description>
</rdf:RDF>



In [ ]:
print(g.serialize(format="json-ld"))

[
  {
    "@id": "http://example.org/carol",
    "https://schema.org/knows": [
      {
        "@id": "http://example.org/bob"
      },
      {
        "@id": "http://example.org/alice"
      }
    ]
  },
  {
    "@id": "http://example.org/bob",
    "https://schema.org/knows": [
      {
        "@id": "http://example.org/carol"
      }
    ]
  },
  {
    "@id": "http://example.org/alice",
    "https://schema.org/knows": [
      {
        "@id": "http://example.org/bob"
      }
    ]
  }
]


In [ ]:
g.parse("https://raw.githubusercontent.com/cursosLabra/pruebawebsem/main/example2.ttl", format="turtle")
print(g.serialize(format="turtle"))

@prefix : <http://example.org/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix schema: <https://schema.org/> .

:alice schema:knows :bob .

:carol schema:birthPlace dbr:Oviedo ;
    schema:knows :alice,
        :bob .

:bob schema:birthPlace dbr:Oviedo ;
    schema:knows :carol .




# SPARQL

La librería rdflib también permite realizar consultas SPARQL

In [ ]:
g.parse("https://raw.githubusercontent.com/cursosLabra/pruebawebsem/main/SPARQL/datos.ttl", format="turtle")

<Graph identifier=N4386f932b84e49308fbc9107e9db2b73 (<class 'rdflib.graph.Graph'>)>

Ejecutar consulta SPARQL

In [ ]:
query1 = """
PREFIX dc:  <http://purl.org/dc/terms/>
PREFIX uni: <http://uniovi.es/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 


SELECT ?p ?c WHERE {
 ?p  dc:creator  ?c .
 ?c  rdf:type    uni:Profesor .
}"""
result = g.query(query1)

Mostrar resultados

In [ ]:
for row in result:
    print(f"{row.p} {row.c}")

http://uniovi.es/biologia http://uniovi.es/ana
http://uniovi.es/quimica http://uniovi.es/ana
http://uniovi.es/biologia http://uniovi.es/juan


# ShEx

[ShEx](http://shex.io/) (Shape Expressions) es un lenguaje para validar datos RDF. La librería [PyShEx](https://github.com/hsolbrig/PyShEx) permite realizar validaciones mediante Python y es compatible con rdflib.

In [ ]:
!pip install pyshex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyshex import ShExEvaluator

In [ ]:
from rdflib import Graph, Namespace

In [ ]:
g = Graph()
g.parse("https://raw.githubusercontent.com/cursosLabra/pruebawebsem/main/ShEx/example1.ttl", format="turtle")


<Graph identifier=Ne8ea743351e44a9bbef00083b5c109b5 (<class 'rdflib.graph.Graph'>)>

In [ ]:
ex = Namespace("http://example.org/")

In [ ]:
shex = """
prefix :       <http://example.org/>
prefix schema: <http://schema.org/> 
prefix xsd:    <http://www.w3.org/2001/XMLSchema#>

:User IRI {
 schema:name  xsd:string   ;
 schema:knows @:User    *
}
"""

In [ ]:
results = ShExEvaluator().evaluate(g, shex, focus = ex.dave, start = ex.User)

In [ ]:
for r in results:
    if r.result:
        print("PASS")
    else:
        print(f"FAIL: {r.reason}")


FAIL:   Testing :dave against shape N5464fb140f5e47c5b44b9cd0b8979a0e
    Datatype mismatch - expected: http://www.w3.org/2001/XMLSchema#string actual: http://www.w3.org/2001/XMLSchema#integer


# SHACL

[SHACL](https://www.w3.org/TR/shacl/) es otro lenguaje para validar datos RDF que puede utilizarse mediante la librería [PySHACL](https://github.com/RDFLib/pySHACL).

In [ ]:
!pip install pyshacl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: prettytable
    Found existing installation: prettytable 0.7.2
    Uninstalling prettytable-0.7.2:
      Successfully uninstalled prettytable-0.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.4.1 requires prettytable<1, but you have prettytable 2.5.0 which is incompatible.


In [ ]:
from pyshacl import validate

In [ ]:
data = Graph()
data.parse("https://raw.githubusercontent.com/cursosLabra/pruebawebsem/main/SHACL/example1.ttl", format="turtle")

<Graph identifier=Nce61830023854e9db0b8173702f46024 (<class 'rdflib.graph.Graph'>)>

In [ ]:
shapes = Graph()
shapes.parse("https://raw.githubusercontent.com/cursosLabra/pruebawebsem/main/SHACL/shapes.ttl", format="turtle")

<Graph identifier=Ndd403f569bc8453ea453ee8c42d5220d (<class 'rdflib.graph.Graph'>)>

In [ ]:
r = validate(data, shacl_graph = shapes)

Running validation in-place, without modifying the DataGraph.
DEBUG:pyshacl.validate:Running validation in-place, without modifying the DataGraph.
Checking if Shape <NodeShape http://example.org/UserShape> defines its own targets.
DEBUG:pyshacl.validate:Checking if Shape <NodeShape http://example.org/UserShape> defines its own targets.
Identifying targets to find focus nodes.
DEBUG:pyshacl.validate:Identifying targets to find focus nodes.
Found 3 Focus Nodes to evaluate.
DEBUG:pyshacl.validate:Found 3 Focus Nodes to evaluate.
Running evaluation of Shape <NodeShape http://example.org/UserShape>
DEBUG:pyshacl.validate:Running evaluation of Shape <NodeShape http://example.org/UserShape>
Running evaluation of Shape <PropertyShape http://example.org/hasName>
DEBUG:pyshacl.validate:Running evaluation of Shape <PropertyShape http://example.org/hasName>
Shape was passed 1 Focus Node/s to evaluate.
DEBUG:pyshacl.validate:Shape was passed 1 Focus Node/s to evaluate.
Running evaluation of Shape <

In [ ]:
conforms, results_graph, results_text = r
print(f"Conforms: {conforms}")
print(results_graph.serialize(format="turtle"))


Conforms: False
@prefix : <http://example.org/> .
@prefix schema1: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :bob ;
            sh:resultMessage "Less than 1 values on :bob->schema1:name" ;
            sh:resultPath schema1:name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinCountConstraintComponent ;
            sh:sourceShape :hasName ],
        [ a sh:ValidationResult ;
            sh:focusNode :carol ;
            sh:resultMessage "Value is not of Node Kind sh:IRI" ;
            sh:resultPath schema1:email ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape :hasEmail ;
            sh:value "carol@mail.org" ] .




# Ontologías

La librería [OWLReady2](https://owlready2.readthedocs.io/) permite trabajar con ontologías [OWL](https://www.w3.org/OWL/) desde Python.

In [ ]:
!pip install owlready2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from owlready2 import get_ontology, sync_reasoner


In [ ]:
onto = get_ontology("https://raw.githubusercontent.com/cursosLabra/pruebawebsem/main/Ontologies/owlExample.xml").load()

Lista de individuos y clases a las que pertenecen sin inferencia

In [ ]:
for i in onto.individuals():
    print(f"Individual: {i} is a: {i.is_a}")

Individual: owlExample.xml.alice is a: [owlExample.xml.Person]
Individual: owlExample.xml.bob is a: [owlExample.xml.Person]
Individual: owlExample.xml.carol is a: [owlExample.xml.Female, owlExample.xml.Person]


Aplicar inferencia

In [ ]:
sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/lib/python3.10/dist-packages/owlready2/hermit:/usr/local/lib/python3.10/dist-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpo5k3au_9
* Owlready2 * HermiT took 1.6736795902252197 seconds
* Owlready * Reparenting owlExample.xml.bob: {owlExample.xml.Person} => {owlExample.xml.Person, owlExample.xml.Male}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


Lista de clases e individuos tras inferencia

In [ ]:
for i in onto.individuals():
    print(f"Individual: {i} is a: {i.is_a}")

Individual: owlExample.xml.alice is a: [owlExample.xml.Person]
Individual: owlExample.xml.bob is a: [owlExample.xml.Person, owlExample.xml.Male]
Individual: owlExample.xml.carol is a: [owlExample.xml.Female, owlExample.xml.Person]
